In [12]:
import json
import pandas as pd
from pathlib import Path
from utils import clean_extracted_data, compute_segmentation_scores

In [13]:
# Load extracted data
output_dir = Path('../../outputs/nv_task/extractions')
outputs = output_dir.glob('*NuExtract*.jsonl')

# Extract substring after NuExtract for each output
nv_task_nu_json = {}
for output in outputs:
    params = output.stem.split('NuExtract-')[1]
    prepend = output.stem.split('NuExtract-')[0].split('_')[-2]

    if 'max_chars=40000' in params:
        continue

    # Excude 1B for now (we know it performs much worse)
    if '1B' in params:
        continue

    key = f'{prepend}_{params}'

    with open(output, 'r') as f:
        nv_task_nu_json[key] = json.load(f)

In [14]:
nv_task_nu_dict = {}
for params, data in nv_task_nu_json.items():
    nv_task_nu_dict[params] = clean_extracted_data(data)


In [15]:
# Load annotations

from labelrepo.projects.nv_task import load_annotations
from utils import get_annotation_summary, run_all_comparisons

annotations = load_annotations()
annotations = annotations[annotations['annotator_name'] != 'alice_chen']

annotations_summary = get_annotation_summary(annotations)


In [16]:
all_scores_df = {}
for params, data in nv_task_nu_dict.items():
    print(f"Running comparisons for {params}")
    scores_df = run_all_comparisons(annotations_summary, data)
    all_scores_df[params] = scores_df

Running comparisons for taskstructured-zeroshot_2-4B_temperature=0.4_do_sample=True
Running comparisons for taskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False
Running comparisons for taskstructured-zeroshot_1_5
Running comparisons for taskstructuredNOMC-zeroshot_2-4B_temperature=0.001_do_sample=True
Running comparisons for taskstructuredNOMC-zeroshot_2-4B_temperature=0
Running comparisons for taskstructuredNOMC-zeroshot_2-4B_temperature=0.4_do_sample=True
Running comparisons for taskstructuredNOMC-zeroshot_v1.5
Running comparisons for taskstructured-zeroshot_2-4B_temperature=0.001_do_sample=True
Running comparisons for taskstructured-zeroshot_2-4B_temperature=0
Running comparisons for taskstructured-noexample-zeroshot_2-4B_temperature=0_do_sample=False
Running comparisons for taskstructuredNOMC-zeroshot_v1.5_window_size=5000


In [17]:
exclude_idx = [_p for _p, v in annotations_summary.items() if v['Exclude'] is not None]
has_task_name = set([_p for _p, v in annotations_summary.items() if v['TaskName'] and ('n/a' not in v['TaskName'])]) - set(exclude_idx)
has_task_noname = set([_p for _p, v in annotations_summary.items() if not(v['TaskName']) or 'n/a' in v['TaskName']]) - set(exclude_idx)

In [18]:
results = {}
nas_per_label = {}
for params, df in all_scores_df.items():
    results[params], nas_per_label[params] = compute_segmentation_scores(df, exclude_idx, has_task_name, has_task_noname)

In [19]:
# Sort by key
results = dict(sorted(results.items()))
for params, df in results.items():
    print(f"Results for {params}")
    display(df)
    display(nas_per_label[params])
    print("\n")

Results for taskstructured-noexample-zeroshot_2-4B_temperature=0_do_sample=False


,n,HasRestingState,Modality,Exclude,TaskName,TaskDescription
All,102,0.16,0.34,0.5,0.69,0.80
No Exclude,94,0.16,0.34,NaN,0.69,0.80
Has Task Name,63,0.06,0.24,NaN,0.82,0.82
Has Task with No Name,31,0.35,0.53,NaN,0.23,0.71


,HasRestingState,Modality,Exclude,TaskName,TaskDescription
All,0.0,0.08,0.96,0.21,0.36
No Exclude,0.0,0.00,1.00,0.14,0.31
Has Task Name,0.0,0.00,1.00,0.00,0.14
Has Task with No Name,0.0,0.00,1.00,0.42,0.65




Results for taskstructured-zeroshot_1_5


,n,HasRestingState,Modality,Exclude,TaskName,TaskDescription
All,103,0.95,0.01,0.0,0.77,0.68
No Exclude,94,0.95,0.01,NaN,0.77,0.68
Has Task Name,63,0.98,0.01,NaN,0.88,0.66
Has Task with No Name,31,0.87,0.00,NaN,0.19,0.80


...

,n,HasRestingState,TaskName,TaskDescription
All,104,0.90,0.80,0.73
No Exclude,95,0.89,0.80,0.73
Has Task Name,64,0.97,0.91,0.71
Has Task with No Name,31,0.74,0.20,0.86


,HasRestingState,TaskName,TaskDescription
All,0.0,0.38,0.50
No Exclude,0.0,0.32,0.45
Has Task Name,0.0,0.14,0.28
Has Task with No Name,0.0,0.68,0.81




Results for taskstructuredNOMC-zeroshot_v1.5_window_size=5000


,n,HasRestingState,TaskName,TaskDescription
All,104,0.89,0.83,0.75
No Exclude,95,0.88,0.83,0.75
Has Task Name,64,0.97,0.92,0.75
Has Task with No Name,31,0.71,0.18,0.79


,HasRestingState,TaskName,TaskDescription
All,0.0,0.49,0.61
No Exclude,0.0,0.44,0.57
Has Task Name,0.0,0.28,0.41
Has Task with No Name,0.0,0.77,0.90


In [20]:
results.keys()

dict_keys(['taskstructured-noexample-zeroshot_2-4B_temperature=0_do_sample=False', 'taskstructured-zeroshot_1_5', 'taskstructured-zeroshot_2-4B_temperature=0', 'taskstructured-zeroshot_2-4B_temperature=0.001_do_sample=True', 'taskstructured-zeroshot_2-4B_temperature=0.4_do_sample=True', 'taskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False', 'taskstructuredNOMC-zeroshot_2-4B_temperature=0', 'taskstructuredNOMC-zeroshot_2-4B_temperature=0.001_do_sample=True', 'taskstructuredNOMC-zeroshot_2-4B_temperature=0.4_do_sample=True', 'taskstructuredNOMC-zeroshot_v1.5', 'taskstructuredNOMC-zeroshot_v1.5_window_size=5000'])

NVExtract 1.5 had high recall, but worse performance than GPT 4o-mini in some fields. 
However it achived a respectable figure for `TaskName` which is one of most critical fields. 

NVExtract 2's JSON generation problems were mostly confined to the 1B model (aside from missing / additional trailing '}')
For the 1B model, removing multi-choice fields helped performance, but for the 4B model, the performance difference was negligible

1B model is worse than 4B

Focusing on 4B vs NUExtract 1.5, 1.5 has lower recall but higher accurac - which is potentially good in this use case

---

In general, it seems examples actually *hurt* performance as it copies the example sometimes (for TaskName and TaskDescription).
Examples helped for multi-choice fields, but it was still not as good as GPT-4o mini on these fields. 
Also, extracing multi choice fields like modality slightly hurt performance on TaskName and TaskDescription.

Thus the most promising option is no multi-choice (NOMC), no example, and temperature = 0.

--- 

Stil haven't tried the 8B model which would be better

## Manual comparison

In [30]:
# Combine all results

subset = ['taskstructured-zeroshot_1_5', 'taskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False']

combined = []
for params, df in nv_task_nu_dict.items():
    if params not in subset:
        continue
    nu = pd.DataFrame(df).T
    nu['annotator_name'] = f'NuExtract{params}'
    nu['pmcid'] = nu['id']
    combined.append(nu)

combined.append(pd.DataFrame(annotations_summary).T)

combined_df = pd.concat(combined)

In [31]:
import pandas as pd
from IPython.display import display

pd.set_option('max_colwidth', 300)
    

def _display(df, pmcids=None, exclude_cols=['pmcid', 'id', 'fMRITasks', 'BehavioralTasks', 'RestingState', 'RestingStateMetadata', 'Condition', 'ContrastDefinition', 'Restingstate', 'TaskDuration', 'TaskDesign']):
    cols = list(set(df.keys()) - set(exclude_cols))
    if pmcids:
        df = df[df.pmcid.isin(pmcids)]
    for _, _df in df.groupby('pmcid'):
        display(_df[cols])

In [32]:
_comp = 'taskstructured-noexample-zeroshot_2-4B_temperature=0_do_sample=False' 

Exclude did not work on NuExtract 1.5 because no example or explanation was given

In [33]:
# Diff on Modality 

# diff_exclude = set(all_scores_df[_comp][all_scores_df[_comp]['Modality'] == False].index.tolist()).union(set(all_scores_df[_comp][all_scores_df[_comp]['Modality'] == False].index.tolist()))
# if diff_exclude:
#     _display(combined_df, list(diff_exclude)[0:2])

In [34]:
# Diff on TaskName

has_taskname_df = combined_df[combined_df.index.isin(has_task_name)]

diff_taskname = set(all_scores_df[_comp][all_scores_df[_comp]['TaskName'] < 0.8].index.tolist()).union(set(all_scores_df[_comp][all_scores_df[_comp]['TaskName'] < 0.8].index.tolist()))

if diff_taskname:
    _display(has_taskname_df, diff_taskname)

,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
2241626,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Fast reproducible identification and large-scale databasing of individual functional cognitive networks,"[Pressing three times the left button with the left thumb button according to visual instructions, Passive viewing of flashing vertical checkerboards, Pressing three times the left button according to visual instruction, Pressing three times the right button according to visual instruction, Pres...","[Right hand action, Vertical checkerboards, Pressing left button, Pressing right button, Pressing left button, Pressing right button, Silently reading, Listening, Solving subtraction problems, Solving subtraction problems]"
2241626,,NuExtracttaskstructured-zeroshot_1_5,[functionalmagneticresonanceimaging(fMRI)],False,"to describe the cerebral inter-subject variability of the correlates of some language, calculation and sensorimotor tasks","[investigate superior fronto-parietal networks, focuses on the inferior frontal and superior temporal lobes]","[mental calculation, language comprehension]"
2241626,None,delavega-aliceoverlap,[fMRI-BOLD],False,NaN,"[In the present research, our goal was to define a simple fMRI test, less than 5 minutes long, that could delineate, in a subject-specific manner, those cerebral circuits. A functional sequence was added to each functional imaging session performed in our lab (Figure ), taking advantage of the ...","[mental calculation task, language comprehension task]"


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
2686646,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,To evaluate the consistency associated with functionally localising reading- and object-sensitive areas of left occipito-temporal cortex.,"[A one-back task with four categories of visual stimuli: written words, pictures of common objects, scrambled pictures of the same objects, and consonant letter strings.]",[Reading]
2686646,,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,investigate intra- and inter-subject consistency for reading and object sensitivity in ventral and lateral OTC respectively,"[visual word recognition, visual object recognition]","[Reading, Object]"
2686646,None,delavega_nv,[fMRI-BOLD],False,NaN,None,[one-back task]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4374765,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,The aim of this study was to investigate the relationship between GABA concentration and fear-related BOLD responses in the insula.,"[The fear provocation paradigm involved presenting still pictures of spiders, of other control animals, generally negative pictures taken from the International Affective Picture System (IAPS), and neutral pictures also obtained from the IAPS. This allowed us to produce a fear-specific contrast ...",[Fear provocation paradigm]
4374765,"participants with current or personal history of psychosis, mood or anxiety disorders",NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,to investigate the relationship between GABA concentration and fear-related BOLD responses in this region,"[presenting still pictures of spiders, of other control animals, generally negative pictures taken from the International Affective Picture System (IAPS, [ ]) and neutral pictures also obtained from the IAPS]",[fear inducing paradigm]
4374765,None,delavega_nv,"[MRS, fMRI-BOLD]",False,NaN,None,[fear inducing paradigm]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4440210,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,fMRI measurements of amygdala activation are confounded by stimulus correlated signal fluctuations in nearby veins draining distant brain regions,[Subjects were shown triplets of geometric shapes (as neutral stimuli) and of threatening scenes as well as fearful faces (as emotional conditions) presented in alternating blocks of neutral and emotional stimuli.],[emotional matching paradigm]
4440210,,NuExtracttaskstructured-zeroshot_1_5,[functionalMRI],True,,[],[]
4440210,None,delavega-aliceoverlap,[fMRI-BOLD],True,NaN,[Subjects were shown triplets of geometric shapes (as neutral stimuli) and of threatening scenes as well as fearful faces (as emotional conditions) presented in alternating blocks of neutral and emotional stimuli.],[emotional matching task]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
5324609,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,The role of the hippocampus in generalizing configural relationships,"[Participants were tested on their ability to generalize what they had learned., Participants learned a set of visual discriminations via trial-and-error.]","[Generalization Test, Learning Phase]"
5324609,,NuExtracttaskstructured-zeroshot_1_5,[],False,examine the brain mechanisms involved in gradually learning and subsequently generalizing a set of visual discriminations,[],[]
5324609,None,delavega-aliceoverlap,"[fMRI-BOLD, fMRI-BOLD]",False,NaN,"[participants learned a set of visual discriminations via trial‐and‐error (learning phase), and were subsequently tested on their ability to generalize what they had learned (generalization phase). Both learning and generalization occurred within a single scanning session and took place in the c...",[acquired equivalence task]


...

,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
9910278,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,We combined established emotion regulation and dietary choice tasks with fMRI to investigate behavioral and neural associations in self-regulation across the two domains in human participants.,"[Participants were asked to either (1) simply view and react naturally or (2) reappraise photographs with different emotional valence., Participants knew that one of these trials would be selected at random and their choice on that trial implemented for real, meaning that they would have to eat ...","[emotion reappraisal, food choice]"
9910278,depression and emotion blindness,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,investigate behavioral and neural associations in self-regulation across the two domains in human participants,"[reappraisal of emotional stimuli, dietary self-control task]","[emotion reappraisal, dietary self-control]"
9910278,None,delavega_nv,[fMRI-BOLD],False,NaN,None,"[emotion reappraisal, dietary self-control]"


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10129386,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,To target the neural correlates of trait self-boundarylessness both in rest and in relation to narrative and minimal self.,"[Participants were administered a set of trait adjectives from the Affective Norms of Emotion Words database. The original 20 positive and 20 negative trait adjectives were translated to Swedish by the first author, slightly modified to make the words more relevant in Swedish, and divided into b...","[SRP, Checking-in]"
10129386,,NuExtracttaskstructured-zeroshot_1_5,[fMRI],True,investigated behavioural and functional magnetic resonance imaging correlates of trait self-boundarylessness during resting state and the performance of two experimental tasks,"[keep their eyes closed and rest but not fall asleep and to not deliberately meditate, administered a set of trait adjectives from the Affective Norms of Emotion Words database, intended to strengthen the sense of perspectival ownership of experience by asking participants to focus on their cent...","[resting state, SRP task, checking-in task]"
10129386,None,delavega_nv,[fMRI-BOLD],True,NaN,"[participants were administered a set of trait adjectives from the Affective Norms of Emotion Words database ( ). The original 20 positive and 20 negative trait adjectives were translated to Swedish by the first author, slightly modified to make the words more relevant in Swedish, and divided in...","[verbal self-referential processing (SRP), checking-in]"


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10597625,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,The neurocognitive processes underlying Pavlovian conditioning in humans are still largely debated.,"[A repeated-measures ANOVA on expectancy ratings performed with trial sequence (6 levels) as a within-subject variable revealed a significant main effect of trial sequence [ F (5, 100) = 57.98, P < 0.001, = 0.744]. A significant linear trend was also identified, indicating that expectanc...","[Expectancy ratings, fMRI univariate whole-brain results, fMRI univariate ROI results, FMRI exploratory FIR results]"
10597625,no significant effects were observed in the bilateral amygdala ROI,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,"to test the prediction that there are dissociable brain networks reflecting the associative strength model and the expectancy model of Pavlovian fear conditioning, consistent with two-system theories of threat and fear reactivity",[to independently localize face processing in the amygdala],"[functional localizer, main task]"
10597625,None,delavega-aliceoverlap,[fMRI-BOLD],False,NaN,[The experimental design was based on the one used by and . The main task consisted of three runs (i.e. functional runs in the scanner) of 45 trials per run for a total of 135 trials across all three runs. Half the trials were CS-alone trials in which the US was omitted. In the other half of ...,[single-cue conditioning paradigm]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10615837,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,"The continuous decline of executive abilities with age is mirrored by increased neural activity of domain-general networks during task processing. So far, it remains unclear how much domain-general networks contribute to domain-specific processes such as language when cognitive demands increase.","[Participants were required to decide whether an auditory stimulus matches with a presented image via yes/no-button press using the index and middle finger of their left hand., Participants were required to decide whether an auditory stimulus matches with a presented image via yes/no-button pres...","[Semantic judgment, Tone judgment]"
10615837,,NuExtracttaskstructured-zeroshot_1_5,[TMS],False,to determine if iTBS could enhance semantic and executive task processing in healthy middle-aged to older adults,"[with varying cognitive demand, to characterize the modulatory effect of pre-SMA stimulation on non-verbal executive demands]","[verbal semantic judgment task, non-verbal tone judgment task]"
10615837,None,delavega_nv,[fMRI-BOLD],False,NaN,"[In both tasks, participants were required to decide whether an auditory stimulus matches with a presented image via yes/no-button press using the index and middle finger of their left hand. The left hand was used to shift motor activity related to the button press to the right hemisphere. The o...","[semantic judgment task, non-verbal tone judgment task]"


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10637045,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,"To investigate central pain mechanisms in the context of the gut-brain axis, novel experimental protocols with painful stimuli from different modalities applied within one brain imaging session, so called ‘multiple threat paradigms’, have more recently been implemented. These were designed to mo...","[Pain arising from the viscera, herein induced by pressure-controlled rectal distensions stimuli, Pain arising from somatically innervated organs such as the muscles or the skin, herein induced using cutaneous thermal heat pain stimuli]","[Visceral pain, Somatic pain]"
10637045,,NuExtracttaskstructured-zeroshot_1_5,"[visceral, somatic]",False,testing pain sensitivity for two pain modalities and employing a multiple-threat paradigm with visceral and somatic pain stimuli for the first time in patients,[],[]
10637045,None,delavega_nv,[fMRI-BOLD],False,NaN,"[On the study day, visceral and somatic pain thresholds were initially determined based on ratings of gradually increasing stimulus intensities, as previously accomplished. Individual pain thresholds served as anchor for the subsequent calibration and matching procedures. During calibration, st...",[multiple-threat paradigm]


It often hallucinates "[Go/No-Go Task, Resting State Scan]" as a task name when given an example (which includes that as an example extraction)

In [35]:
# Diff on TaskDescription

has_taskname_df = combined_df[combined_df.index.isin(has_task_name)]

diff_desc = set(all_scores_df[_comp][all_scores_df[_comp]['TaskDescription'] < 0.8].index.tolist()).union(set(all_scores_df[_comp][all_scores_df[_comp]['TaskDescription'] < 0.8].index.tolist()))

if diff_taskname:
    _display(has_taskname_df, diff_desc)

,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
2241626,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Fast reproducible identification and large-scale databasing of individual functional cognitive networks,"[Pressing three times the left button with the left thumb button according to visual instructions, Passive viewing of flashing vertical checkerboards, Pressing three times the left button according to visual instruction, Pressing three times the right button according to visual instruction, Pres...","[Right hand action, Vertical checkerboards, Pressing left button, Pressing right button, Pressing left button, Pressing right button, Silently reading, Listening, Solving subtraction problems, Solving subtraction problems]"
2241626,,NuExtracttaskstructured-zeroshot_1_5,[functionalmagneticresonanceimaging(fMRI)],False,"to describe the cerebral inter-subject variability of the correlates of some language, calculation and sensorimotor tasks","[investigate superior fronto-parietal networks, focuses on the inferior frontal and superior temporal lobes]","[mental calculation, language comprehension]"
2241626,None,delavega-aliceoverlap,[fMRI-BOLD],False,NaN,"[In the present research, our goal was to define a simple fMRI test, less than 5 minutes long, that could delineate, in a subject-specific manner, those cerebral circuits. A functional sequence was added to each functional imaging session performed in our lab (Figure ), taking advantage of the ...","[mental calculation task, language comprehension task]"


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
3078751,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,investigate the role of the caudate nucleus in events that violate predictions,"[watched movies of a dancer producing the same sequences either according to the cue (88%) or not (12%), participants watched a dancer performing according to cue, but occasionally making mistakes]","[movement observation paradigm, behavioral probe session]"
3078751,one participant had to be excluded from further analyses due to insufficient performance (below 2 SDs from mean),NuExtracttaskstructured-zeroshot_1_5,[],False,investigates whether caudate nucleus signals for breaches of expectation in a movement observation paradigm,"[contrasted invalidly cued switches and invalidly cued repetitions vs. validly cued switches and validly cued repetitions, parametrically modeled the first dissimilar transition cue after 4, 3, 2, or 1 identical transition cue(s), modeled by comparing switch cues to repetition cues]","[violation hypothesis (i), saliency hypothesis (ii), change hypothesis (iii)]"
3078751,None,delavega-other,[fMRI-BOLD],False,NaN,"[participants had to register auditory cues that determined what movement the dancer was to perform next, and watch the ensuing movements, participants watched a dancer performing according to cue, but occasionally making mistakes. Previous to playing the task, the participants were instructed o...",[movement observation paradigm]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4115625,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,To investigate gender differences on voice localizer scans by employing the conventional univariate analysis as well as MVPA.,"[Subjects were instructed to close their eyes and passively listen to a large variety of sounds. Stimuli were presented in a simple block design and divided into vocal (20 blocks) and non-vocal (20 blocks) conditions., Multivariate pattern classification was performed on unsmoothed and non-norma...","[Voice localizer paradigm, Multivariate pattern analysis]"
4115625,,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,to localize voice-related areas in both female and male listeners and to investigate whether brain maps may differ depending on the gender of the listener,[known to give robust estimates of the temporal voice areas (TVAs)],[voice localizer]
4115625,None,delavega-other,[fMRI-BOLD],False,NaN,"[The voice localizer includes a variety of different vocal sounds, not exclusively female or male voices, but also speech and non-speech of women, men and infants and non-vocal sounds (e.g., environmental sounds), Subjects were instructed to close their eyes and passively listen to a large varie...",[Voice localizer paradigm]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
6024199,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Neural responses to emotional involuntary memories in posttraumatic stress disorder: Differences in timing and activity,"[Twenty-one individuals with PTSD and 21 non-PTSD, trauma-exposed controls performed an involuntary memory task, while undergoing a functional magnetic resonance imaging scan. Environmental sounds served as cues for well-associated pictures of negative and neutral scenes. We used a finite impuls...","[Involuntary memory task, Post-scan recall]"
6024199,unpaired sounds,NuExtracttaskstructured-zeroshot_1_5,[functionalmagneticresonanceimaging(fMRI)],False,examined the spatiotemporal dynamics of the neural activity underlying negative and neutral involuntary memory retrieval,[while undergoing a functional magnetic resonance imaging scan],[involuntary memory task]
6024199,None,delavega-other,"[fMRI-BOLD, fMRI-BOLD]",False,NaN,"[During the re-encoding runs, each sound-picture pair and unpaired sound was presented. Participants rated on a 7-point scale how emotional the stimuli were. During the recall runs, the 55 paired and 40 unpaired sounds (randomly intermixed) were presented, panned 15° to either the left or the ri...",[involuntary memory]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
6137311,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Differential effects of parent and peer presence on neural correlates of risk taking in adolescence,"[A computerized driving task where participants could make safe or risky decisions, in the presence of a peer and their parent., A widely used task examining risk-taking at the behavioral and neural level.]","[Yellow Light Game (YLG), Stoplight Task]"
6137311,,NuExtracttaskstructured-zeroshot_1_5,[],False,compare within-person peer and parent influences on risky decision-making,[a computerized driving task],[Yellow Light Game (YLG)]
6137311,None,delavega_nv,[fMRI-BOLD],False,NaN,"[The YLG (Op de Macks et al ., ) is an adaption of the widely used Stoplight Task (Gardner and Steinberg, ; Chein et al ., ;) that examines risk-taking at the behavioral and neural level. In the YLG, participants were asked to drive a virtual car from the driver’s point of view along a ...",[Yellow Light Game (YLG)]


...

,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10637045,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,"To investigate central pain mechanisms in the context of the gut-brain axis, novel experimental protocols with painful stimuli from different modalities applied within one brain imaging session, so called ‘multiple threat paradigms’, have more recently been implemented. These were designed to mo...","[Pain arising from the viscera, herein induced by pressure-controlled rectal distensions stimuli, Pain arising from somatically innervated organs such as the muscles or the skin, herein induced using cutaneous thermal heat pain stimuli]","[Visceral pain, Somatic pain]"
10637045,,NuExtracttaskstructured-zeroshot_1_5,"[visceral, somatic]",False,testing pain sensitivity for two pain modalities and employing a multiple-threat paradigm with visceral and somatic pain stimuli for the first time in patients,[],[]
10637045,None,delavega_nv,[fMRI-BOLD],False,NaN,"[On the study day, visceral and somatic pain thresholds were initially determined based on ratings of gradually increasing stimulus intensities, as previously accomplished. Individual pain thresholds served as anchor for the subsequent calibration and matching procedures. During calibration, st...",[multiple-threat paradigm]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10656574,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Neural tracking of social hierarchies in adolescents’ real-world social networks,"[Adolescents viewed yearbook photos of their peers from their school and grade. The task was created using E-prime 2.0, and one version was made for each grade level within each school.]",[Classmates fMRI task]
10656574,15 7th grade participants from one school were excluded due to sociometric data calculation errors,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,examine adolescents’ neural tracking of peers from their real-world social network that varied in social preferences and popularity,[task-based fMRI face recognition task adapted from ],[Classmates fMRI task]
10656574,None,delavega_nv,[fMRI-BOLD],False,NaN,"[The task had four conditions: high social preference (i.e. z -score between 1 and 5 on social preference), low social preference (i.e. z -score between −1 and −5 on social preference), high popularity (e.g. z-score between 1 and 5 on popularity) and low popularity (i.e. z -score betwee...",[Classmates task]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10791126,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,To assess the causal effect of VVGs on the behavioral and neural correlates of empathy and emotional reactivity to violence.,"[Participants either received electric stimuli themselves or saw images of the confederate indicating that he was currently receiving electric stimulation. The stimuli were either painful or perceptible but not painful., To investigate emotional reactivity to violent images, we used an affective...","[Empathy-for-pain task, Emotional reactivity task]"
10791126,participants with prior VVG experience,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,to assess the causal effect of VVGs on the behavioral and neural correlates of empathy and emotional reactivity to violence,"[In trials of the Self condition, participants passively received electrical stimuli. In the Other condition, participants experienced how another person (a confederate) received electrical stimuli., Participants were presented pictures with different content (violent or neutral) and different c...","[Empathy-for-pain task, Emotional reactivity task]"
10791126,None,delavega_nv,[fMRI-BOLD],False,NaN,"[ In trials of the Self condition, participants passively received electrical stimuli. In the Other condition, participants experienced how another person (a confederate) received electrical stimuli. The stimuli were either painful or not painful. In the cue phase, an arrow indicated the recipie...","[Empathy-for-pain task, Emotional reactivity task]"


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10990450,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,This study examined whether perceived friendship quality was related to better mental health and lower neural stress response in young people with childhood adversity (CA).,[A well-validated and widely used acute psychosocial stress paradigm for fMRI.],[Montreal Imaging Stress Task]
10990450,"individuals were eligible to participate if they were aged between 16 and 26 years, able to speak, write, and understand English, had a body mass index (BMI) between 18.5 and 29.9µkg/m , did not currently take medication (e.g. corticosteroids) likely to compromise the interpretation of our data,...",NuExtracttaskstructured-zeroshot_1_5,[functionalmagneticresonanceimaging(fMRI)],False,This study examined whether perceived friendship quality was related to better mental health and lower neural stress response in young people with CA.,[a well-validate acute psychosocial stress paradigm combining the stress-eliciting effects of high cognitive demands (solving math problems under time pressure) with negative social feedback (on screen and verbally via the experimenter)],[Montreal Imaging Stress Task (MIST)]
10990450,None,delavega_nv,[fMRI-BOLD],False,NaN,"[During the 5 min stress condition ( A), participants were asked to answer math problems of varying difficulty under time constraints whilst receiving trial-by-trial on screen performance feedback (‘correct’ in green, ‘error’ in red, or ‘timeout’ in yellow). To answer, participants were prov...",[Montreal Imaging Stress Task (MIST)]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
11078806,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Motivated with joy or anxiety: Does approach-avoidance goal framing elicit differential reward-network activation in the brain?,"[Participants were instructed that they could win or lose points during the stopwatch task depending on their performance, and their overall goal was to earn the total point of zero or larger than zero at the end of the experiments.]",[stopwatch task]
11078806,,NuExtracttaskstructured-zeroshot_1_5,[],False,examine whether and how the reward network in the brain is involved in approach versus avoidance goals,"[a game-like, intrinsically motivating task]",[stopwatch task]
11078806,None,delavega_nv,[fMRI-BOLD],False,NaN,"[For each trial, participants were presented with a stopwatch that automatically started; they were asked to presss a button to stop the stopwatch within 50 ms of the 5-s time point. If participants stopped the stopwatch within this time window, the trial was deemed a success. In contrast, if pa...",[stopwatch task]


In [36]:
## Studies with no task name

no_taskname_df = combined_df[combined_df.index.isin(has_task_noname)]

In [37]:
_display(no_taskname_df)

,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
3825257,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,The default modes of reading: modulation of posterior cingulate and medial prefrontal cortex connectivity associated with comprehension and task focus while reading,"[Reading is a fundamental human capacity and yet it can easily be derailed by the simple act of mind-wandering., Reading is a fundamental human capacity and yet it can easily be derailed by the simple act of mind-wandering.]","[Reading, Reading]"
3825257,,NuExtracttaskstructured-zeroshot_1_5,[rs-fMRI],True,to understand why the DMN could be implicated in both costs and benefits to reading,[],[]
3825257,None,delavega_nv,[fMRI-BOLD],True,NaN,None,[]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4110030,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,To investigate sex differences and menstrual cycle effects in resting state functional connectivity of fronto-parietal cognitive control networks.,[Participants were instructed to relax and keep their eyes closed during scanning.],[Resting state fMRI]
4110030,no use of hormonal contraceptives or other hormone regulating medicaments currently or for the last six months,NuExtracttaskstructured-zeroshot_1_5,[fMRI],True,"To address this issue, the present study focuses on sex differences and investigates whether resting state fMRI is stable in men and women or changes within relative short-term periods (i.e., across the menstrual cycle).",[],[]
4110030,None,delavega_nv,[fMRI-BOLD],True,NaN,None,[]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4386762,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,"To assess whether the coupling between ALFF and FC depends on anatomical brain localization, frequency band, and pathological conditions.","[Voxel-wise connectivity analysis was carried out using 3dTcorrMap (AFNI package, [ ]). For each voxel, Pearson correlation coefficients between each voxel and all other voxels of the brain were computed; then, after a z-Fisher transformation, these coefficients were averaged., ALFF measures vox...","[Functional connectivity, Amplitude of low frequency fluctuations]"
4386762,,NuExtracttaskstructured-zeroshot_1_5,[functionalmagneticresonanceimaging(fMRI)],True,assess their relationship in healthy elderly as well as the relevant changes induced by Alzheimer’s Disease (AD) and Mild Cognitive Impairment (MCI),[],[]
4386762,None,delavega_nv,[fMRI-BOLD],True,NaN,None,[]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4488375,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Shaped by the Past: The Default Mode Network Supports Cognition that Is Independent of Immediate Perceptual Input,"[Participants had to encode the identity of shapes presented on screen and when prompted by a coloured slide to respond based on the position of a specific target shape on the prior trial., Participants either made decisions about the location of shapes as they are presented on screen or with re...","[1-back, 0-back]"
4488375,,NuExtracttaskstructured-zeroshot_1_5,[fMRI],False,to confirm previous findings,"[observing non-coloured shapes presented at fixation waiting for the presentation of a coloured slide at which point they would indicate which side of the fixation cross a target shape was, encode the identity of shapes presented on screen and when prompted by a coloured slide to respond based o...","[0-back, 1-back]"
4488375,None,delavega_nv,[fMRI-BOLD],False,NaN,[The task used in this experiment was programmed using PsychoPy [ ]. The task featured a 0-back and a 1-back condition that continuously switched from one another throughout the experimental session (see ). Our paradigm is broadly similar to the paradigm used by Smallwood and colleagues [ ] and...,[n/a]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
4526228,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,To determine how the level of physical fitness (measured as CRF) and PA (measured via accelerometer) are related to functional brain health measured as SD .,[We collected resting functional magnetic resonance BOLD data from 100 healthy older participants (60–80 years).],[Resting state fMRI]
4526228,,NuExtracttaskstructured-zeroshot_1_5,[MRI],True,to determine how the level of physical fitness (measured as CRF) and PA (measured via accelerometer) are related to functional brain health measured as SD,[],[]
4526228,None,delavega_nv,"[fMRI-BOLD, DiffusionMRI]",True,NaN,None,[]


...

,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
8975992,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Processing of visual and non-visual naturalistic spatial information in the 'parahippocampal place area',"[The movie stimulus offers ecologically more valid visual stimulation than a paradigm using blocks of pictures., The audio-description stimulus offers an exclusively auditory stimulation.]","[movie stimulus, audio-description stimulus]"
8975992,,NuExtracttaskstructured-zeroshot_1_5,"[visual, auditory]",False,"investigate the perception of scene-related, spatial information embedded in two naturalistic stimuli","[visual spatial perception, non-visual spatial perception]","[movie, audio-description]"
8975992,None,delavega_nv,[fMRI-BOLD],False,NaN,None,[n/a]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
9202476,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts’ properties of the DAN and VAN.","[In this computerized task, subjects were presented with a complex figure in the middle of the screen. Then, a few patterns were shown in the periphery, from which one was matched with the presented pattern. In the first trials, two patterns were presented in the periphery, and it was increased ...",[match-to-sample (MTS) task]
9202476,,NuExtracttaskstructured-zeroshot_1_5,[MRI],False,"evaluate correlation of sleep quality with visual attention and search, functional, and tracts’ properties of the DAN and VAN",[visual search and attention],[match-to-sample (MTS) task]
9202476,None,delavega-aliceoverlap,"[fMRI-BOLD, DiffusionMRI]",True,NaN,None,[]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
9729227,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,Re-examined the VAN and the DAN neuroanatomy by co-registering individual network maps in a common functional space.,[Used 110 7 T resting-state functional MRI datasets from the Human Connectome Project S1200.],[Resting-state functional imaging (rs-fMRI)]
9729227,,NuExtracttaskstructured-zeroshot_1_5,[],False,,[],[]
9729227,None,delavega_nv,[fMRI-BOLD],True,NaN,None,[]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10028637,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,True,"To ask whether the connectivity within and between three networks that are primary sites of propofol‐induced sedation and key to conscious cognition—the DAN, ECN, and DMN—underlie individual responsiveness differences under anaesthesia.","[Participants were asked to perform a computerized auditory target detection task (50 trials), which aimed to assess individual responsiveness differences during moderate anaesthesia., Participants underwent two functional scans during wakefulness and moderate anaesthesia. A plot‐driven auditory...","[Auditory target detection task, Resting‐state fMRI]"
10028637,,NuExtracttaskstructured-zeroshot_1_5,[MagneticResonanceImaging],True,to understand the brain mechanisms of consciousness,[measured behavioural responsiveness],[target detection task]
10028637,None,delavega_nv,[fMRI-BOLD],True,NaN,None,[n/a]


,Exclude,annotator_name,Modality,HasRestingState,StudyObjective,TaskDescription,TaskName
10031743,NaN,NuExtracttaskstructuredNOMC-noexample-zeroshot_2-4B_temperature=0_do_sample=False,NaN,False,To explore the neural correlates of four distinct dimensions of Formal Thought Disorder (FTD) using perfusion and structural MRI.,[],[]
10031743,substance abuse other than nicotine,NuExtracttaskstructured-zeroshot_1_5,[3Tmagneticresonanceimaging(MRI)],True,to develop specific treatments for FTD,[],[]
10031743,None,delavega_nv,"[StructuralMRI, DiffusionMRI, fMRI-CBF]",True,NaN,None,[]


In [29]:
## Focus on studies where NuExtract 1.5 did not extract task name, but NuExtract 2-4B did